### Task 2

#### Task 2.1

In [ ]:
# Load data (csv file)
data <- read.csv("transformer_data.csv", header = TRUE, sep = ",")
# Display the first few rows of the data
head(data)

,time,Y,Ta,S,I
,<int>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,23.49673,7.083333,0,15.89954
2,2,22.72033,7.116667,0,15.86177
3,3,22.05270,7.216667,0,16.12147
4,4,21.53805,7.250000,0,15.44763
5,5,20.99503,7.316667,0,16.25832
6,6,20.61153,7.483333,0,16.63265


In [ ]:
# Plot data

png("2_zoom_out_plot_dual_axis.png", width = 12, height = 6, units = "in", res = 300)

# Plot main variables (temperature in °C)
par(bg = "white", mar = c(5, 4, 4, 4) + 0.1)  # Leave space on right for second y-axis
plot(data$Y, type = "l", col = "blue", xlab = "Time (h)", ylab = "Temperature (°C)", 
     main = "Transformer Data with Solar Radiance", 
     ylim = range(c(data$Y, data$Ta, data$I)), lwd = 2)
lines(data$Ta, col = "springgreen2", lwd = 2)
lines(data$I, col = "magenta2", lwd = 2)

# Add solar radiance (S) on a secondary axis
par(new = TRUE)
plot(data$S, type = "l", col = "goldenrod2", axes = FALSE, xlab = "", ylab = "", lwd = 2)
axis(side = 4, col = "goldenrod2", col.axis = "goldenrod2")
mtext("Solar Radiance (W/m²)", side = 4, line = 3, col = "goldenrod2")

# Legend
legend("topright", 
       legend = c("Y (°C)", "Ta (°C)", "I (°C)", "S (W/m²)"), 
       col = c("blue", "springgreen2", "magenta2", "goldenrod2"), 
       lty = 1, lwd = 2)

dev.off()


pdf 
  2

### Task 2.2

In [ ]:
# Functions Exercise 2
kf_logLik_dt <- function(par, df) {
  # par: vector of parameters
  # df: data frame with observations and inputs as columns (Y, Ta, S, I)
  # par: Could be on the form c(A11, A12, A21, A22, B11, B12, B21, B22, Q11, Q12, Q22)
  A   <- matrix() # transition matrix
  B   <- matrix() # input matrix
  Sigma1lt <- matrix() # lower-triangle of system covariance matrix
  Sigma1   <- Sigma1lt %*% t(Sigma1lt) # THAT IS!!! The system covariance matrix is given by Qlt %*% t(Qlt) (and is this symmetric positive definite)
  C   <- matrix() # observation matrix
  Sigma2 <- matrix() # observation noise covariance matrix
  X0  <- matrix() # initial state

  # Variables
  obs_cols <- c("Y") # observation column names
  input_cols <- c("Ta","S","I") # input column names

  # pull out data
  Y  <- as.matrix(df[, obs_cols])     # m×T
  U  <- as.matrix(df[, input_cols])   # p×T
  Tn <- nrow(df)

  # init
  n      <- nrow(A)
  x_est  <- matrix(Y[1,], n, 1)            # start state from first obs
  x_est <- X0 
  P_est  <- diag(1e1, n)                   # X0 prior covariance
  logLik <- 0

  for (t in 1:Tn) {
    # prediction step
    x_pred <- # write the prediction step
    P_pred <- # write the prediction step (Sigma_xx)

    # innovation step
    y_pred  <- # predicted observation
    S_t     <- # predicted observation covariance (Sigma_yy)
    innov   <- # innovation (one-step prediction error)

    # log-likelihood contribution
    logLik <- logLik - 0.5*(sum(log(2*pi*S_t)) + t(innov) %*% solve(S_t, innov))

    # update step
    K_t    <- # Kalman gain
    x_est  <- # reconstructed state
    P_est  <- 1 # reconstructed covariance
  }

  as.numeric(logLik)
}




In [ ]:
# Optimizer wrapper
estimate_dt <- function(start_par, df, lower=NULL, upper=NULL) {
  negLL <- function(par){ -kf_logLik_dt(par, df) }
  optim(
    par    = start_par, fn = negLL,
    method = "L-BFGS-B",
    lower  = lower, upper = upper,
    control= list(maxit=1000, trace=1)
  )
}